# Querying eLuxembourgensia
This notebook will illustrate how to use the eLuxembourgensia digital collection. 
First, we will request the user (you) to enter a name of a person. We will then query Wikidata for the person. 
If there are multiple entries, then the user will be requested to select the desired result. 
We will then use the birth and death dates of the selected person to query the eLuxembourgensia collection and display
the newspapers that were published during that person's lifetime.



In [1]:
# A function to fetch the Wikidata API
import requests

def fetch_wikidata(params):
    url = 'https://www.wikidata.org/w/api.php'
    try:
        return requests.get(url, params=params)
    except:
        return 'ERR'

In [2]:
# Request a name from the user
name = input("Enter a name to search for in Wikidata:")

In [ ]:
# Build the wikidata parameters
params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': name,
        'language': 'en'
    }

# Fetch the data from Wikidata
data = fetch_wikidata(params)
data

# Convert the response to JSON
if data!= 'ERR':
    data = data.json()

# for testing purposes, show the data retrieved    
data

{'searchinfo': {'search': 'Paul Eyschen'},
 'search': [{'id': 'Q945406',
   'title': 'Q945406',
   'pageid': 895526,
   'concepturi': 'http://www.wikidata.org/entity/Q945406',
   'repository': 'wikidata',
   'url': '//www.wikidata.org/wiki/Q945406',
   'display': {'label': {'value': 'Paul Eyschen', 'language': 'en'},
    'description': {'value': 'prime minister of Luxembourg (1841-1915)',
     'language': 'en'}},
   'label': 'Paul Eyschen',
   'description': 'prime minister of Luxembourg (1841-1915)',
   'match': {'type': 'label', 'language': 'en', 'text': 'Paul Eyschen'}},
  {'id': 'Q13104511',
   'title': 'Q13104511',
   'pageid': 14648442,
   'concepturi': 'http://www.wikidata.org/entity/Q13104511',
   'repository': 'wikidata',
   'url': '//www.wikidata.org/wiki/Q13104511',
   'display': {'label': {'value': 'Monument Paul Eyschen', 'language': 'en'},
    'description': {'value': 'monument in Stadtbredimus, Luxembourg',
     'language': 'en'}},
   'label': 'Monument Paul Eyschen',
  